In [2]:
import cv2
from fpdf import FPDF
import cv2.aruco as aruco
import numpy as np
from glob import glob

In [3]:
# Define the parameters for the Charuco board
num_squares_x = 5
num_squares_y = 7
square_length = 100
marker_length = 80

# Create the Charuco board
board = aruco.CharucoBoard_create(
    squaresX=num_squares_x,
    squaresY=num_squares_y,
    squareLength=square_length,
    markerLength=marker_length,
    dictionary=aruco.Dictionary_get(aruco.DICT_4X4_50)
)

In [6]:

# Save the Charuco board image
board_image = board.draw(
    (num_squares_x * square_length, num_squares_y * square_length))
cv2.imwrite("charuco_board.png", board_image)

True

In [8]:

# Convert the image to PDF
pdf = FPDF(unit="mm", format='A4')
pdf.add_page()
pdf.image("charuco_board.png", x=(210-175)//2, y=(297-245)//2, w=175, h=245)
pdf.output("charuco_board.pdf")

''

In [11]:
# Load the calibration images
calibration_images = glob("../../bilderserien/charuco/*/*.jpg")

In [26]:
# Detect Charuco markers in the calibration images
all_corners = []
all_ids = []
for i in calibration_images:
    image = cv2.imread(i)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, board.dictionary)
    if len(corners) >= 6:
        ret, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            corners, ids, gray, board)
        if ret and len(charuco_corners) >= 6:
            all_corners.append(charuco_corners)
            all_ids.append(charuco_ids)

In [31]:
len(all_corners)

79

In [27]:
# Calibrate the camera
camera_matrix = np.zeros((3, 3))
distortion_coefficients = np.zeros((1, 5))
ret, camera_matrix, distortion_coefficients, _, _ = aruco.calibrateCameraCharuco(
    all_corners, all_ids, board, gray.shape[::-1], camera_matrix, distortion_coefficients)

# Print the camera matrix and distortion coefficients
print("Camera Matrix:")
print(camera_matrix)
print("Distortion Coefficients:")
print(distortion_coefficients)

Camera Matrix:
[[3.38490391e+03 0.00000000e+00 2.33240156e+03]
 [0.00000000e+00 3.39258948e+03 1.34515747e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
[[ 0.00244731  0.35281624  0.00568091  0.00252201 -0.66625855]]


In [33]:
np.savetxt("camera_matrix.csv", camera_matrix, delimiter=",")
np.savetxt("distortion_coefficients.csv",
           distortion_coefficients, delimiter=",")